In [3]:
import numpy as np

In [16]:
class LiftedProductCode:

    #Initialize with cyclic group order n (size of the lifting group G).
    def __init__(self, group_order):
        self.n = group_order
        self.G = self._generate_group()

    #Generate the cyclic group matrices G = [I, P, P^2, ..., P^(n-1)].
    def _generate_group(self):
        P = np.roll(np.eye(self.n), shift=1, axis=1)
        return [np.linalg.matrix_power(P, i) for i in range(self.n)]

    #Lift a base matrix by replacing entries with group matrices or zero blocks.
    def lifted_matrix(self, base_matrix):
        lifted_rows = []

        for row in base_matrix:
            block_row = []
            for val in row:
                if isinstance(val, int) and val != 0:
                    block = self.G[val % self.n]
                else:
                    block = np.zeros((self.n, self.n))
                block_row.append(block)
            lifted_rows.append(np.hstack(block_row))

        return np.vstack(lifted_rows)

    #Compute the lifted product code for matrices A and B.
    def compute_product_code(self, A, B):
        A_lift = self.lifted_matrix(A)
        B_lift = self.lifted_matrix(B)

        mA, nA = A_lift.shape
        mB, nB = B_lift.shape

        # Construct H_X
        HX_left = np.kron(A_lift, np.eye(mB))
        HX_right = np.kron(np.eye(mA), B_lift)
        HX = np.hstack((HX_left, HX_right))

        # Construct H_Z
        HZ_left = np.kron(np.eye(nA), B_lift.T)
        HZ_right = np.kron(A_lift.T, np.eye(nB))
        HZ = np.hstack((HZ_left, HZ_right))

        # Check CSS condition
        if np.all((HX @ HZ.T) % 2 == 0):
            print("CSS condition satisfied.")
            return HX, HZ
        else:
            print("CSS condition NOT satisfied.")
            return None, None


In [10]:
# Example: 1 (3-bit repetition code)
A = [[0, 1, 0],
    [0, 1, 1]]

lpc = LiftedProductCode(group_order=3)

HX, HZ = lpc.compute_product_code(A, A)

if HX is not None:
    print("H_X shape:", HX.shape)
    print("H_Z shape:", HZ.shape)


CSS condition satisfied.
H_X shape: (36, 108)
H_Z shape: (81, 108)


In [15]:
# Example: 2 (Hamming Code)
B = np.matrix([[1, 0, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1], [0, 0, 1, 0, 1, 1, 1]], dtype =int)
lpc = LiftedProductCode(group_order=7)
HX, HZ = lpc.compute_product_code(A, B)

if HX is not None:
    print("H_X shape:", HX.shape)
    print("H_Z shape:", HZ.shape)

CSS condition satisfied.
H_X shape: (294, 539)
H_Z shape: (147, 539)
